In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [74]:
#https://api.tidesandcurrents.noaa.gov/api/prod/

url = 'https://api.tidesandcurrents.noaa.gov/api/prod/'
parameters = 'datagetter?date=latest&station=9410170&product=one_minute_water_level&datum=STND&time_zone=lst&units=english&format=json'

r = requests.get(url + parameters)

r.status_code

200

In [75]:
r.json()

{'metadata': {'id': '9410170',
  'name': 'San Diego',
  'lat': '32.7156',
  'lon': '-117.1767'},
 'data': [{'t': '2025-02-03 23:41', 'v': '8.904'}]}

In [76]:
request_json = r.json()['metadata']
list = []

for k in request_json:
    list.append(request_json[k])

print(list)

['9410170', 'San Diego', '32.7156', '-117.1767']


In [77]:
parameters = 'datagetter?begin_date=20250110&end_date=20250112&station=9410170&product=one_minute_water_level&datum=STND&time_zone=lst&units=english&format=csv'
r = requests.get(url + parameters)

r.status_code

200

In [78]:
csv_file = open('test.csv', 'wb')
csv_file.write(r.content)
csv_file.close()

In [79]:
paramters = 'datagetter?date=latest&station=9410170&product=one_minute_water_level&datum=STND&time_zone=lst&units=english&format=json'


### Use Selenium for dynamic rendered websites

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [4]:
#use the path you saved it in
PATH = r"C:\Program Files\chromedriver-win64\chromedriver.exe"

URL = "https://tidesandcurrents.noaa.gov/stations.html?type=Water+Levels"

In [82]:
service = Service(executable_path=PATH)

driver = webdriver.Chrome(service=service)

In [83]:
STATION_ID = [] #grab all stations that take water level measurements


In [84]:
driver.get(URL)

wait = WebDriverWait(driver, 10)  # stay on site for 10s

a_elements = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, '//a[@style="color: #015FA9;"]'))
)

In [85]:
for element in a_elements:
    # regex for extracting 7-digit station id
    match = re.search(r'waterlevels\.html\?id=(\d{7})', element.get_attribute('href'))
    if match:
        STATION_ID.append(match.group(1))

In [97]:
print(len(STATION_ID)) #test if correctly collected
print(STATION_ID[0])
print(STATION_ID[len(STATION_ID) - 1])
print(STATION_ID)

#store station ids
with open('stationid.txt', 'w') as text_file:
    for id in STATION_ID:
        text_file.write(id + '\n')


292
8735180
9449880
['8735180', '8735391', '8735523', '8736897', '8737048', '8737138', '8738043', '8739803', '9450460', '9451054', '9451600', '9452210', '9452400', '9452634', '9453220', '9454050', '9454240', '9455090', '9455500', '9455760', '9455920', '9457292', '9457804', '9459450', '9459881', '9461380', '9461710', '9462450', '9462620', '9463502', '9464212', '9468333', '9468756', '9491094', '9497645', '2695535', '2695540', '9410170', '9410230', '9410660', '9410840', '9411340', '9412110', '9413450', '9414290', '9414523', '9414750', '9414863', '9415020', '9415102', '9415144', '9416841', '9418767', '9419750', '9751364', '9751381', '9751401', '9751639', '9752235', '9752695', '9755371', '9759110', '9759394', '9759938', '8461490', '8465705', '8467150', '8551762', '8551910', '8555889', '8557380', '8594900', '8720030', '8720218', '8720219', '8720226', '8720357', '8721604', '8722670', '8722956', '8723214', '8723970', '8724580', '8725114', '8725520', '8726384', '8726520', '8726607', '8726674', 

In [87]:
driver.quit()

### Now we can use their api with our list of station IDs


In [12]:
# for help formatting data collection
# https://tidesandcurrents.noaa.gov/api-helper/url-generator.html

url_toaa = 'https://api.tidesandcurrents.noaa.gov/api/prod/'
parameters = 'datagetter?begin_date=20200101&end_date=20201231&station=8735180&product=hourly_height&datum=STND&time_zone=lst&units=english&format=csv'

In [24]:
#dates in format: yyyymmdd
#product can be: hourly_height
#default local standard time
def callAPI(begin_date, end_date, station, product, directory_path):
    #get metadata first
    r = requests.get(url_toaa + 'datagetter?begin_date=20200101&end_date=20200101&station=' + station + '&product=hourly_height&datum=STND&time_zone=lst&units=english&format=json')
    if (r.status_code != 200):
        return 'failed'

    #format: 'id','name', 'lat', 'lon'
    data = r.json()['metadata']
    metadata = []
    for k in data:
        metadata.append(data[k])

    #get actual data
    parameters = 'datagetter?begin_date=' + begin_date + '&end_date=' + end_date + '&station=' + station + '&product=' + product + '&datum=STND&time_zone=lst&units=english&format=csv'
    
    r = requests.get(url_toaa + parameters)

    if (r.status_code != 200):
        return 'failed'
    
    year = begin_date[:4]
    #save to file
    csv_file = open(directory_path + year + '_' + metadata[0] + '_' + metadata[1] + '_' + metadata[2] + '_' + metadata[3] + '.csv', 'wb')
    csv_file.write(r.content)
    csv_file.close()

    return r


In [90]:
#TESTING ONLY
# status = callAPI('20200101', '20201231', str(STATION_ID[0]), 'hourly_height')

if (status == 'failed'):
    print('failed')

In [ ]:
with open('stationid.txt', 'w') as text_file:
    for id in STATION_ID:
        text_file.write(id + '\n')

In [30]:
#copied for use
STATION_ID = ['8735180', '8735391', '8735523', '8736897', '8737048', '8737138', '8738043', '8739803', '9450460', '9451054', '9451600', '9452210', '9452400', '9452634', '9453220', '9454050', '9454240', '9455090', '9455500', '9455760', '9455920', '9457292', '9457804', '9459450', '9459881', '9461380', '9461710', '9462450', '9462620', '9463502', '9464212', '9468333', '9468756', '9491094', '9497645', '2695535', '2695540', '9410170', '9410230', '9410660', '9410840', '9411340', '9412110', '9413450', '9414290', '9414523', '9414750', '9414863', '9415020', '9415102', '9415144', '9416841', '9418767', '9419750', '9751364', '9751381', '9751401', '9751639', '9752235', '9752695', '9755371', '9759110', '9759394', '9759938', '8461490', '8465705', '8467150', '8551762', '8551910', '8555889', '8557380', '8594900', '8720030', '8720218', '8720219', '8720226', '8720357', '8721604', '8722670', '8722956', '8723214', '8723970', '8724580', '8725114', '8725520', '8726384', '8726520', '8726607', '8726674', '8726724', '8727520', '8728690', '8729108', '8729210', '8729840', '8670870', '8679598', '9044020', '9044030', '9044036', '9044049', '9063020', '9063028', '9063038', '9063053', '9063063', '9063079', '9063085', '9063090', '9075002', '9075014', '9075035', '9075065', '9075080', '9075099', '9087023', '9087031', '9087044', '9087057', '9087068', '9087072', '9087077', '9087088', '9087096', '9052000', '9052030', '9052058', '9052076', '9034052', '9099004', '9099018', '9099044', '9099064', '9099090', '9063007', '9063009', '9063012', '9014070', '9014080', '9014087', '9014090', '9014098', '8311030', '8311062', '9076024', '9076027', '9076033', '9076060', '9076070', '1611400', '1612340', '1612401', '1612480', '1615680', '1617433', '1617760', '8760721', '8760922', '8761305', '8761724', '8761927', '8761955', '8762075', '8762482', '8764044', '8764227', '8764314', '8766072', '8767816', '8767961', '8768094', '8410140', '8411060', '8413320', '8418150', '8419870', '8570283', '8571421', '8571892', '8573364', '8573927', '8574680', '8575512', '8577330', '8443970', '8447386', '8447435', '8447636', '8447930', '8449130', '8741533', '8747437', '8531680', '8534720', '8536110', '8537121', '8539094', '8510560', '8516945', '8518750', '8518962', '8651370', '8652587', '8654467', '8656483', '8658120', '8658163', '9431647', '9432780', '9435380', '9437540', '9439040', '9439099', '9439201', '1619910', '1630000', '1631428', '1770000', '1820000', '1890000', '8540433', '8545240', '8546252', '8548989', '8452660', '8452944', '8454000', '8454049', '8661070', '8665530', '8770475', '8770520', '8770613', '8770777', '8770808', '8770822', '8770971', '8771013', '8771341', '8771367', '8771450', '8771486', '8771972', '8772471', '8772985', '8773037', '8773146', '8773259', '8773701', '8773767', '8774230', '8774770', '8775132', '8775222', '8775237', '8775241', '8775283', '8775296', '8775792', '8776139', '8776604', '8777812', '8778490', '8779280', '8779748', '8779749', '8779770', '8631044', '8632200', '8635027', '8635750', '8636580', '8637689', '8638610', '8638901', '8639348', '9440083', '9440357', '9440422', '9440569', '9440581', '9440910', '9441102', '9442396', '9443090', '9444090', '9444900', '9445958', '9446484', '9447130', '9449424', '9449880']
station_count = len(STATION_ID)

In [25]:
#get hourly water levels for 0th station
def getDecade(stat_id, directory_path):
    for year in range(2004, 2024 + 1):
        status = callAPI(str(year) + '0101', str(year) + '1231', stat_id, 'hourly_height', directory_path)

        if (status == 'failed'):
            print('failed at year: ' + year)
            break

In [99]:
# callAPI('20200101', '20201231', str(STATION_ID[0]), 'hourly_height')

In [ ]:
import os

for i, station in enumerate(STATION_ID):
    print('starting ' + station)
    directory_path = 'data/' + station +'/'
    os.makedirs(directory_path, exist_ok=True)
    getDecade(station, directory_path)
    print('finishing ' + station + ' ' + str(i + 1) + '/' + str(station_count))

starting 8735180
finishing 8735180 0/292
starting 8735391


KeyboardInterrupt: 

KeyboardInterrupt: 